# Projeto de Análise de Desigualdade em Saúde no modelo HEAT/OMS
## Bloco 2025/1 = análise do sistema de mortalidade do SUS com agregação por unidades federativas




#### Aspectos gerais e participantes

Projeto Desenvolvido em parceria da MS/SECTICS/DESID com a Organização Mundial de Saúde

Elaborado em linguagem Python, utilizando a biblioteca pysus para acesso ao SIM (Sistema de Informações de Mortalidade do SUS)

Coordenador-geral:

Coordenadores: 

Cientista de dados: Marcos Mesquita

Outros participantes:

Data: 06/12/2024


#### Objetivos

O principal objetivo desse projeto é realizar a avaliação de desigualdade na saúde no Brasil com ênfase para a dimensão de unidades subnacionais, estados e municípios brasileiros, a partir da aplicação do modelo do HEAT, tendo como objetivos específicos:

- Apresentar indicadores de saúde no Brasil desagregados por subunidades nacionais em uma visualização interativa a partir do modelo do HEAT Plus;
- Oferecer um repositório comum e atualizado com dados de desigualdade da saúde no Brasil desagregados por estados e municípios em um modelo sob reconhecimento internacional;
- Permitir exploração e acesso dos dados e da metodologia para outros tipos de análises.

# Etapa 1 - Definição do Bloco

O projeto ocorrerá em blocos sucessivos, nos quais serão definidos o problema e os indicadores a serem analisados.

### Definição do problema do bloco



 H0 - as unidades federativa são variáveis determinantes para mortalidade no país

### Escopo do Bloco 1


* indicador: mortalidade (número de mortes por população, dados de morte do SIM/SUS e dados populacionais pelo Censos de 2010 e 2022)
* localização: todo o Brasil, com agregação de mortalidade por Estados e Municípios
* período: 2010 e 2022 
* doenças: “doenças evitáveis” conforme definido pelo OCDE
* outros critérios de filtro: até 74 anos de idade (em razão da aplicação do critério de "doenças evitáveis")

# Etapa 2 - Carga primária dos dados por bloco

## Aspectos teóricos e técnicos das fontes


#### Aspectos técnicos - ambiente, linguagem e ferramentas principais

A análise será feita utilizando a linguagem Python e a biblioteca Pysus que permite acesso às principais bases do SUS de forma direta.
O Pysus exige um ambiente linux para execução de alguns comandos o que foi possível, mesmo em uma máquina Windows, a partir do WSL do Windows.
Foi criado um ambiente em WSL e o conjunto da extração, tratamento e análise dos dados foi realizado no formato Notebook em um editor VSCode, dentro de um Dev Container: Default Linux Universal.


#### Morte e mortalidade no HEAT

Para entendermos como o HEAT trata a mortalidade, fizemos uma busca de indicadores de mortalidade no WHO Health Inequality Data Repository.

O repositório traz dados e metadados de todos os indicadores utilizados pelo HEAT.
Apresenta um total de 3915 indicadores, distribuídos entre 14 seções.

Foi feita uma revisão geral dos indicadores relacionados à mortalidade, que depois foi refinada pela busca das seguintes palavras : “death”, “mortality”, “fatal”, ‘fatality” ,“Probability of dying”, “suicide”. E foram encontrados 388 indicadores que trazem uma destas expressões na sua descrição.


#### SIM (Sistema de Informações de Mortalidade do SUS)

A mortalidade e os indicadores de morte serão buscados no SIM (Sistema de Informações de Mortalidade do SUS).
O SIM disponibiliza dados de mortes no Brasil desde 1996 por ocorrências, gerido pelo Departamento de Análise de Situação de Saúde, da Secretaria de Vigilância em Saúde do Ministério da Saúde, em conjunto com as Secretarias Estaduais e Municipais de Saúde. Estas últimas responsáveis efetivamente pela coleta dos dados.

O SIM traz informações do indivíduo, como sexo, idade, localidade e causas da morte, classificadas de acordo com o CID (Classificação Internacional de Doenças e Problemas Relacionadas à Saúde).


#### CID (Classificação Internacional de Doenças e Problemas Relacionadas à Saúde)

Com o intuito de padronizar a nomenclatura de patologias e com abrangência para todo o mundo, foi elaborado pela Organização Mundial da Saúde (OMS) uma  classificação internacional de doenças, a CID (Classificação Internacional de Doenças e Problemas Relacionadas à Saúde). 

A Classificação é atualizada periodicamente. A versão mais atual é a CID11, publicada em janeiro de 2022.

No entanto, por se tratar de revisão que traz impactos nos registros e que exige considerável atualização das ferramentas de captura das informações, o SIM utiliza versões anteriores da CID.
- CID9 - dados até 1995
- CID10 - a partir de 1996 até a data atual

Em razão do recorte temporal dos dados, de 1998 a 2022, iremos utilizar somente a CID10.
A CID10 é organizada em 22 capítulos e possui, além da classificação de doenças, descrição de códigos que permitem identificar sinais, sintomas, queixas, causas externas e circunstâncias sociais.


#### CID de doenças evitáveis

Como elemento do escopo deste trabalho iremos tratar de doenças evitáveis.

O Brasil, a partir da Secretaria de Vigilância em Saúde apresenta uma classificação de doenças evitáveis. 

Lista de causas evitáveis pode ser encontrada nos links: http://tabnet.datasus.gov.br/cgi/sim/Obitos_Evitaveis_0_a_4_anos.pdf




#### População (Censo)


Os dados de população são os do Censo, acessados pelo site do IBGE, https://sidra.ibge.gov.br/Tabela/9606






## Instalação e importação de bibliotecas

#### Instalação de bibliotecas

In [1]:
# Costuma ser exigida a instalação destas bibliotecas no primeiro acesso ao Pysus
!apt-get update  
!apt-get install libffi-dev  # Install libffi-dev
!pip install --upgrade pip  # Ensure pip is up to date
!pip install pysus # Exa
!pip install gdown
!pip install pylance
!pip install tabulate

Reading package lists... Done
E: List directory /var/lib/apt/lists/partial is missing. - Acquire (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.3.1
    Uninstalling pip-24.3.1:
      Successfully uninstalled pip-24.3.1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 3.5 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ..

#### Importação de bibliotecas

In [2]:
# importar biblioteca básicas para manipulação de dados
import pandas as pd
import numpy as np
import os
import sidrapy
import requests # para carga de endereços web - api
import gdown
import openpyxl
from tabulate import tabulate

In [3]:
# importar bibliotecas para carga dos dados
import pysus
# import pylance
from pysus import SIM
from pysus.online_data.SIM import download
from pysus.preprocessing.decoders import translate_variables_SIM
from pysus.preprocessing.SIM import group_and_count, redistribute_missing, redistribute_cid_chapter
from pysus.online_data.SIM import get_CID9_table, get_CID10_table, get_municipios, get_ocupations
from ftplib import FTP
from pathlib import Path  

In [3]:
# Para organizar resultados, gerar prints em amarelo

def print_y(text):
  """Imprime o texto fornecido na cor amarela.

  Args:
    text: O texto a ser impresso.
  """
  print(f"\033[33m ==> {text}\033[0m")

# Exemplo de uso:
print_y("Este texto será impresso em amarelo!")
print("Este em normal!")

 ==> Este texto será impresso em amarelo!
Este em normal!


#### Diretórios - configuração

In [ ]:
# Definir diretório padrão para pysus
pysus.online_data.SIM.CACHEPATH="./files_in_sim"
# Definir pasta para download de arquivos csv e parquet
import os 

# Define diretório para armazenar arquivos do sim (carregados por pysus) quando o SIM.CACHEPATH não funcionar
os.makedirs('files_in_sim', exist_ok=True)  
downloads_sim = "downloads_sim_original"

# Define diretório para armazenar outros arquivos
os.makedirs('files_in_geral', exist_ok=True)  

# Define diretório para armazenar arquivos temporários
os.makedirs('files_temp', exist_ok=True) 

# Define diretório para exportar arquivos finais ou de relatórios
os.makedirs('files_out', exist_ok=True) 

#### Acesso ao SIM

In [4]:
sim = SIM().load() # Loads the files from DATASUS

In [5]:
sim.paths

(/dissemin/publicos/SIM/CID10/DORES, /dissemin/publicos/SIM/CID9/DORES)

In [6]:
sim.groups

{'CID10': 'DO', 'CID9': 'DOR'}

## Arquivo principal - DO

DO - Declaração de Óbito, conforme acessado no SIM

#### Carga SIM principal

In [32]:
# Carrega os dados do SIM para o ano de 2010
# Definição de variáveis
grupo = "CID10" # Agrupamento por CID10
uf = "" # A sigla BR carrega dados de todos as unidades federativas
ano = [2010] # Anos do escopo, 2010 e 2022
local = downloads_sim # Local para armazenar os arquivos, definido nas etapas anteriores

# sim.get_files("CID10", uf="BR", year=2010)
dobr_2010 = download(grupo,uf,ano,downloads_sim) # Carrega os dados do SIM
# dobr2010 = download("CID10","BR",2010,'TESTE_EXCLUIR') # Carrega os dados do SIM
# dobr2010 

DOTO2010.parquet: 100%|██████████| 16.6k/16.6k [00:00<00:00, 51.3kB/s]


In [8]:
# Carrega os dados do SIM para os demais anos
# Definição de variáveis
grupo = "CID10" # Agrupamento por CID10
uf = "BR" # A sigla BR carrega dados de todos as unidades federativas
ano = [2019,2020,2021,2022] # Anos do escopo, 2010 e 2022
local = downloads_sim # Local para armazenar os arquivos, definido nas etapas anteriores

# sim.get_files("CID10", uf="BR", year=2010)
dobr2010 = download(grupo,uf,ano,downloads_sim) # Carrega os dados do SIM
# dobr2010 = download("CID10","BR",2010,'TESTE_EXCLUIR') # Carrega os dados do SIM
# dobr2010 

0it [00:00, ?it/s]


In [ ]:
# dodf = dobr.to_dataframe()

### Tratamento do arquivo principal

O dicionário do SIM está em "./Estrutura_SIM_para_CD.pdf"

Trabalhamos o CID presente no campo CAUSABAS.

Em razão do escopo da População Estimada (que apresenta dados por Município, mas não de outras variáveis de desigualdade) a primeira versão será um recorte da base DOBR com os dados de CID ('CAUSABAS'), idade (a partir do tratamento de 'DTNASC'), e de Município (a partir do tratamento de 'CODMUNRES').

Por se tratar de doenças evitáveis, o primeiro filtro a ser feito é com base na idade de óbito, que deve ser abaixo de 75 anos.

A idade de óbito está presente no campo IDADE. Ela é composta de 3 dígitos, o primeiro identifica se a contagem é por dias(1), semanas(2), meses (3), ano (4), mais de 100 anos (5).

Em razão disto, vamos desconsiderar todos os valores de dígito 4 e mais de 75 (475).


Iremos tratar do arquivo de 2010 separado dos outros anos pela especificade daquele arquivo.



In [4]:
# Acessar os dados do SIM de 2010 como 1 dataframe

# Definir o diretório onde os arquivos estão localizados
directory = 'files_in_sim'

# Listar todos os arquivos na pasta
file_list = os.listdir(directory)

# Filtrar apenas os arquivos parquet
parquet_files = [file for file in file_list if file.endswith('2010.parquet')]

# Inicializar uma lista para armazenar os dataframes
dataframes = []

# Iterar sobre os arquivos parquet e ler cada um em um dataframe
for file in parquet_files:
    file_path = os.path.join(directory, file)
    df = pd.read_parquet(file_path)
    dataframes.append(df)

# Concatenar todos os dataframes em um único dataframe
dobr_10_temp = pd.concat(dataframes, ignore_index=True)

# Exibir informações do dataframe combinado
dobr_10_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1136947 entries, 0 to 1136946
Data columns (total 60 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   CONTADOR    1133938 non-null  object
 1   ORIGEM      1136947 non-null  object
 2   TIPOBITO    1136947 non-null  object
 3   DTOBITO     1136947 non-null  object
 4   HORAOBITO   1136947 non-null  object
 5   NATURAL     1136947 non-null  object
 6   DTNASC      1136947 non-null  object
 7   IDADE       1136947 non-null  object
 8   SEXO        1136947 non-null  object
 9   RACACOR     1136947 non-null  object
 10  ESTCIV      1136947 non-null  object
 11  ESC         1136947 non-null  object
 12  OCUP        1136947 non-null  object
 13  CODMUNRES   1136947 non-null  object
 14  CODBAIRES   1136947 non-null  object
 15  LOCOCOR     1136947 non-null  object
 16  CODESTAB    1136947 non-null  object
 17  CODMUNOCOR  1136947 non-null  object
 18  CODBAIOCOR  1136947 non-null  object
 19  

In [5]:
# Excluir valores nulos da coluna CAUSABAS
dobr_10_temp01 = dobr_10_temp.dropna(subset=['CAUSABAS'])
dobr_10_temp01.shape[0]


1136947

In [6]:
# Criar nova coluna 'OBITO_SOMA'
dobr_10_temp01['OBITO_SOMA'] = np.where(dobr_10_temp01['CONTADOR'].isna(), dobr_10_temp01['contador'], dobr_10_temp01['CONTADOR']).astype(int)

# Exibir as primeiras linhas do DataFrame para verificar a nova coluna
dobr_10_temp01.head()

,CONTADOR,ORIGEM,TIPOBITO,DTOBITO,HORAOBITO,NATURAL,DTNASC,IDADE,SEXO,RACACOR,...,DTRECEBIM,UFINFORM,CB_PRE,MORTEPARTO,DTCADINF,TPOBITOCOR,DTCADINV,contador,NUMERODN,OBITO_SOMA
0,1,1,2,28042010,0500,,09031925,485,2,4,...,08102010,,I64,,,,,NaN,NaN,1
1,2,1,2,13042010,0800,,30031966,444,1,4,...,05052010,,I461,,,,,NaN,NaN,2
2,3,1,2,11032010,0815,815,12071913,496,2,4,...,22122010,,R99,,,,,NaN,NaN,3
3,4,1,2,22082010,2230,815,25031970,440,1,4,...,02092010,,X959,,,,,NaN,NaN,4
4,5,1,2,28112010,1250,,27091963,447,1,4,...,27012011,,R99,,,,,NaN,NaN,5


In [7]:
dobr_10_temp01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1136947 entries, 0 to 1136946
Data columns (total 61 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   CONTADOR    1133938 non-null  object
 1   ORIGEM      1136947 non-null  object
 2   TIPOBITO    1136947 non-null  object
 3   DTOBITO     1136947 non-null  object
 4   HORAOBITO   1136947 non-null  object
 5   NATURAL     1136947 non-null  object
 6   DTNASC      1136947 non-null  object
 7   IDADE       1136947 non-null  object
 8   SEXO        1136947 non-null  object
 9   RACACOR     1136947 non-null  object
 10  ESTCIV      1136947 non-null  object
 11  ESC         1136947 non-null  object
 12  OCUP        1136947 non-null  object
 13  CODMUNRES   1136947 non-null  object
 14  CODBAIRES   1136947 non-null  object
 15  LOCOCOR     1136947 non-null  object
 16  CODESTAB    1136947 non-null  object
 17  CODMUNOCOR  1136947 non-null  object
 18  CODBAIOCOR  1136947 non-null  object
 19  

In [8]:
# Agrupar dobr_10_temp02 por 'DTOBITO', 'CAUSABAS', 'CODMUNRES', 'IDADE', 'SEXO', 'RACACOR' e somar 'CONTADOR'
dobr_10_temp02 = dobr_10_temp01.groupby(['DTOBITO', 'CAUSABAS', 'CODMUNRES', 'IDADE', 'SEXO', 'RACACOR'])['OBITO_SOMA'].count().reset_index()


# mostrar dados gerais da tabela após transformações
tab = dobr_10_temp02
print(f'Registro da tabela : {tab.shape[0]}')
print(f'Contador de óbitos : {tab.OBITO_SOMA.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql', maxcolwidths=50))

Registro da tabela : 1134923
Contador de óbitos : 1136947
+----+-----------+------------+-------------+---------+--------+-----------+--------------+
|    |   DTOBITO | CAUSABAS   |   CODMUNRES |   IDADE |   SEXO |   RACACOR |   OBITO_SOMA |
|----+-----------+------------+-------------+---------+--------+-----------+--------------|
|  0 |  01012010 | A060       |   231380    |   432   |   1    |   4       |            1 |
|  1 |  01012010 | A09        |   210340    |   401   |   2    |   4       |            1 |
|  2 |  01012010 | A09        |   220500    |   493   |   1    |   4       |            1 |
|  3 |  01012010 | A09        |   220770    |   468   |   2    |   2       |            1 |
|  4 |  01012010 | A09        |   220780    |   489   |   1    |   1       |            1 |
+----+-----------+------------+-------------+---------+--------+-----------+--------------+


In [9]:
# Arquivo para doenças evitáveis - excluir maiores de 74 anos
dobr_10_temp03 = dobr_10_temp02.copy()

# Substituir valores '     ' e '-' por '0' na coluna IDADE
dobr_10_temp03['IDADE'] = dobr_10_temp03['IDADE'].replace(['     '], '0')

# transformar IDADE em inteiro e excluir valores maiores que 474
dobr_10_temp03['IDADE'] = dobr_10_temp03['IDADE'].astype(int)

dobr_10_temp03.shape[0]


1134923

In [ ]:
# Filtrar as idades menores de 75
dobr_10_temp04 = dobr_10_temp03[dobr_10_temp03['IDADE'] < 475].reset_index(drop=True)

# mostrar dados gerais da tabela após transformações
tab = dobr_10_temp04
print(f'Registro da tabela : {tab.shape[0]}')
print(f'Contador de óbitos : {tab.OBITO_SOMA.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql', maxcolwidths=50))

Registro da tabela : 721140
Contador de óbitos : 722229
+----+-----------+------------+-------------+---------+--------+-----------+--------------+
|    |   DTOBITO | CAUSABAS   |   CODMUNRES |   IDADE |   SEXO |   RACACOR |   OBITO_SOMA |
|----+-----------+------------+-------------+---------+--------+-----------+--------------|
|  0 |  01012010 | A060       |   231380    |     432 |   1    |   4       |            1 |
|  1 |  01012010 | A09        |   210340    |     401 |   2    |   4       |            1 |
|  2 |  01012010 | A09        |   220770    |     468 |   2    |   2       |            1 |
|  3 |  01012010 | A09        |   290080    |     401 |   1    |   4       |            1 |
|  4 |  01012010 | A09        |   290320    |     307 |   1    |   4       |            1 |
+----+-----------+------------+-------------+---------+--------+-----------+--------------+


In [15]:
# Salvamento provisório
# dobr_19a22_temp01.to_parquet('dobr_19a22_temp01.parquet')

In [12]:
# Carregar e tratar arquivos de DO de outros anos

# Read the parquet files
dobr2019 = pd.read_parquet('files_in_sim/DOBR2019.parquet')
dobr2020 = pd.read_parquet('files_in_sim/DOBR2020.parquet')
dobr2021 = pd.read_parquet('files_in_sim/DOBR2021.parquet')
dobr2022 = pd.read_parquet('files_in_sim/DOBR2022.parquet')

# Concatenate the dataframes
dobr_19a22_temp = pd.concat([dobr2019,dobr2020,dobr2021,dobr2022], ignore_index=True)

In [16]:
print(dobr_19a22_temp.info())
dobr_19a22_temp.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6283540 entries, 0 to 6283539
Data columns (total 87 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   ORIGEM      object
 1   TIPOBITO    object
 2   DTOBITO     object
 3   HORAOBITO   object
 4   NATURAL     object
 5   CODMUNNATU  object
 6   DTNASC      object
 7   IDADE       object
 8   SEXO        object
 9   RACACOR     object
 10  ESTCIV      object
 11  ESC         object
 12  ESC2010     object
 13  SERIESCFAL  object
 14  OCUP        object
 15  CODMUNRES   object
 16  LOCOCOR     object
 17  CODESTAB    object
 18  ESTABDESCR  object
 19  CODMUNOCOR  object
 20  IDADEMAE    object
 21  ESCMAE      object
 22  ESCMAE2010  object
 23  SERIESCMAE  object
 24  OCUPMAE     object
 25  QTDFILVIVO  object
 26  QTDFILMORT  object
 27  GRAVIDEZ    object
 28  SEMAGESTAC  object
 29  GESTACAO    object
 30  PARTO       object
 31  OBITOPARTO  object
 32  PESO        object
 33  TPMORTEOCO  object
 34  OBITOGRAV   object

,ORIGEM,TIPOBITO,DTOBITO,HORAOBITO,NATURAL,CODMUNNATU,DTNASC,IDADE,SEXO,RACACOR,...,FONTES,TPRESGINFO,TPNIVELINV,NUDIASINF,DTCADINF,MORTEPARTO,DTCONCASO,FONTESINF,ALTCAUSA,CONTADOR
0,1,2,21062019,0230,829,292990,12041941,478,2,2,...,,,,,,,,,,950245
1,1,2,01052019,1327,829,292740,22061956,462,1,2,...,,,,,,,,,,950246
2,1,2,16052019,1855,829,292260,05041940,479,2,5,...,,,,,,,,,,950247
3,1,2,03052019,0640,831,317010,18051950,468,2,1,...,,,,,,,,,,950248
4,1,2,26062019,0840,850,500630,13061959,460,1,1,...,,,,,,,,,,950249


In [14]:
# Filtrar campos
dobr_19a22_temp01 = dobr_19a22_temp[['DTOBITO','CAUSABAS', 'CODMUNRES','IDADE','SEXO','RACACOR','CONTADOR']]

dobr_19a22_temp01.shape[0]

6283540

In [18]:
contador_nulo = dobr_19a22_temp01.CONTADOR.count()
contador_nulo

6283540

In [18]:
# Agrupar dobr_10_temp02 por 'DTOBITO', 'CAUSABAS', 'CODMUNRES', 'IDADE', 'SEXO', 'RACACOR' e somar 'CONTADOR'
dobr_19a22_temp02 = dobr_19a22_temp01.groupby(['DTOBITO', 'CAUSABAS', 'CODMUNRES', 'IDADE', 'SEXO', 'RACACOR'])['CONTADOR'].count().reset_index()

dobr_19a22_temp02.rename(columns={'CONTADOR':'OBITO_SOMA'} , inplace=True)

# mostrar dados gerais da tabela após transformações
tab = dobr_19a22_temp02
print(f'Registro da tabela : {tab.shape[0]}')
print(f'Contador de óbitos : {tab.OBITO_SOMA.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql', maxcolwidths=50))

Registro da tabela : 6252305
Contador de óbitos : 6283540
+----+-----------+------------+-------------+---------+--------+-----------+--------------+
|    |   DTOBITO | CAUSABAS   |   CODMUNRES |   IDADE |   SEXO |   RACACOR |   OBITO_SOMA |
|----+-----------+------------+-------------+---------+--------+-----------+--------------|
|  0 |  01012019 | A049       |     210820  |     477 |      2 |         4 |            1 |
|  1 |  01012019 | A049       |     355080  |     452 |      1 |         1 |            1 |
|  2 |  01012019 | A09        |     150553  |     479 |      1 |         4 |            1 |
|  3 |  01012019 | A09        |     310620  |     478 |      2 |         4 |            1 |
|  4 |  01012019 | A09        |     320520  |     467 |      1 |         1 |            1 |
+----+-----------+------------+-------------+---------+--------+-----------+--------------+


In [19]:
columns = dobr_19a22_temp02.columns

for column in columns:
    count_nulo = dobr_19a22_temp02[column].isna().sum()
    print(f"Coluna {column} tem {count_nulo} valores nulos")

Coluna DTOBITO tem 0 valores nulos
Coluna CAUSABAS tem 0 valores nulos
Coluna CODMUNRES tem 0 valores nulos
Coluna IDADE tem 0 valores nulos
Coluna SEXO tem 0 valores nulos
Coluna RACACOR tem 0 valores nulos
Coluna OBITO_SOMA tem 0 valores nulos


In [22]:
# transformar IDADE em inteiro 
dobr_19a22_temp03 = dobr_19a22_temp02.copy()
dobr_19a22_temp03['IDADE'] = dobr_19a22_temp03['IDADE'].astype(int)

In [31]:
# Excluir valores maiores que 474
dobr_19a22_temp04 = dobr_19a22_temp03[dobr_19a22_temp03['IDADE'] < 475]

# mostrar dados gerais da tabela após transformações
tab = dobr_19a22_temp04
print(f'Registro da tabela : {tab.shape[0]}')
print(f'Contador de óbitos : {tab.OBITO_SOMA.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql', maxcolwidths=50))

Registro da tabela : 3706335
Contador de óbitos : 3724693
+----+-----------+------------+-------------+---------+--------+-----------+--------------+
|    |   DTOBITO | CAUSABAS   |   CODMUNRES |   IDADE |   SEXO |   RACACOR |   OBITO_SOMA |
|----+-----------+------------+-------------+---------+--------+-----------+--------------|
|  1 |  01012019 | A049       |     355080  |     452 |      1 |         1 |            1 |
|  4 |  01012019 | A09        |     320520  |     467 |      1 |         1 |            1 |
|  5 |  01012019 | A09        |     330040  |     472 |      1 |         1 |            1 |
|  7 |  01012019 | A09        |     350970  |     460 |      1 |         4 |            1 |
|  9 |  01012019 | A09        |     355100  |     452 |      2 |         1 |            1 |
+----+-----------+------------+-------------+---------+--------+-----------+--------------+


In [27]:
# # Carga provisória
# dobr_19a22_temp04.to_parquet('dobr_19a22_temp04.parquet')
# dobr_10_temp04 = pd.read_parquet('dobr_10_temp04.parquet')

In [28]:
# Concatenate the dataframes
dobr_10_19a22_temp = pd.concat([dobr_19a22_temp04,dobr_10_temp04], ignore_index=True)
dobr_10_19a22_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4427475 entries, 0 to 4427474
Data columns (total 7 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   DTOBITO     object
 1   CAUSABAS    object
 2   CODMUNRES   object
 3   IDADE       int64 
 4   SEXO        object
 5   RACACOR     object
 6   OBITO_SOMA  int64 
dtypes: int64(2), object(5)
memory usage: 236.5+ MB


In [29]:
# Resultado

shape1 = dobr_10_19a22_temp.CAUSABAS.count()

print_y(f" O arquivo de declarações de óbito dos anos de 2010,2019,2020,2021 e 2022 tem o total de {shape1} registros ")


 ==>  O arquivo de declarações de óbito dos anos de 2010,2019,2020,2021 e 2022 tem o total de 4427475 registros 


In [30]:
# Ajustar os campos
dobr_10_19a22_temp01 = dobr_10_19a22_temp.copy()

# assegurar CODMUNRES como string
dobr_10_19a22_temp01['CODMUNRES'] = '_' + dobr_10_19a22_temp01['CODMUNRES'].astype(str).str[:6]

# campo ano_obito formado pelos 4 últimos caracteres de DTOBITO, como número
dobr_10_19a22_temp01['ANO_OBITO'] = '_' + dobr_10_19a22_temp01['DTOBITO'].str[-4:]

# excluir colunas DTOBITO e IDADE
dobr_10_19a22_temp02 = dobr_10_19a22_temp01.drop(['DTOBITO','IDADE'], axis=1)

# mostrar dados gerais da tabela após transformações
tab = dobr_10_19a22_temp02
print(f'Registro da tabela : {tab.shape[0]}')
print(f'Soma de óbitos : {tab.OBITO_SOMA.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registro da tabela : 4427475
Soma de óbitos : 4446922
+----+------------+-------------+--------+-----------+--------------+-------------+
|    | CAUSABAS   | CODMUNRES   |   SEXO |   RACACOR |   OBITO_SOMA | ANO_OBITO   |
|----+------------+-------------+--------+-----------+--------------+-------------|
|  0 | A049       | _355080     |      1 |         1 |            1 | _2019       |
|  1 | A09        | _320520     |      1 |         1 |            1 | _2019       |
|  2 | A09        | _330040     |      1 |         1 |            1 | _2019       |
|  3 | A09        | _350970     |      1 |         4 |            1 | _2019       |
|  4 | A09        | _355100     |      2 |         1 |            1 | _2019       |
+----+------------+-------------+--------+-----------+--------------+-------------+


#### Resultados e geração de arquivo DOBR

##########       Etapa diferenciada da versão      ##########

Atenção na hora de salvar arquivo temporário com as variáveis de versão, formato e local.

In [34]:
# Salvar arquivo em parquet
dobr_10_19a22_temp02.to_parquet('files_in_geral/dobr_10_19a22_u75.parquet')

In [18]:
# Contador

# apresentar valor da diferença entre os números de registros de dobr e dobr_filtered

shape_2010 = dobr_10_temp.shape[0]
shape_2019 = dobr2019.shape[0]
shape_2020 = dobr2020.shape[0]
shape_2021 = dobr2021.shape[0]
shape_2022 = dobr2022.shape[0]
shape_all = shape_2010 + shape_2019 + shape_2020 + shape_2021 + shape_2022

print_y(f"""O número de registros de óbitos originais por ano:
        * 2010: {shape_2010}
        * 2019: {shape_2019}
        * 2020: {shape_2020}
        * 2021: {shape_2021}
        * 2022: {shape_2022}
        """)


dif = shape_all - dobr_10_19a22_temp02.shape[0]
print(f"\033[33m ==> O número de registros excluídos por apresentar idade maior de 74 anos foi de {dif} registros\033[0m")
print(f"\033[33m ==> O número de registros de óbitos resultantes é de {dobr_10_19a22_temp02.shape[0]} registros\033[0m")

 ==> O número de registros de óbitos originais por ano:
        * 2010: 1136947
        * 2019: 1349801
        * 2020: 1556824
        * 2021: 1832649
        * 2022: 1544266
        
 ==> O número de registros excluídos por apresentar idade maior de 74 anos foi de 2973565 registros
 ==> O número de registros de óbitos resultantes é de 4446922 registros


## Carga SIM auxiliar

O SIM traz algumas tabelas auxiliares de CID10 e municípios. 

Não iremos trazemos estas informações, no entanto, porque iremos utilizar dados de municípios concomitante à população do Censo do IBGE e sobre os CIDs iremos consultar a informações já da lista de doenças evitáveis, conforme OCDE.

In [ ]:
# # Download de tabela auxiliar de CID10
# cid10 = get_CID10_table()
# cid10.head()

In [ ]:
# # Download de tabela auxiliar de CID10
# munic = get_municipios()
# print(munic.head())

## Carga externa ao SIM

Iremos carregar informações de fontes externas sobre população e CIDs de doenças evitáveis.

##### Carga de CIDs doenças evitáveis - SVS

O arquivos de doenças evitáveis foi tratado a partir de documento da OCDE e da SVS e será dada carga neles pelo google drive.

In [19]:
# # Download de tabela tratada de CIDs evitáveis de acordo com documentos da SVS

# url = 'https://docs.google.com/spreadsheets/d/1YWurDqnBDMz4ACOEctY9uuQkEm7uSRAogzZETQrLCXk/edit?usp=drive_link'
# # Convert the Google Sheets URL to a CSV export URL
# csv_url = url.replace('/edit?usp=drive_link', '/export?format=csv')

# # Read the CSV data into a pandas DataFrame
# df = pd.read_csv(csv_url)

# # Now you can work with the DataFrame 'df'
# df.head()


In [20]:
# # Salvar arquivo csv
# df.to_csv('downloads_outros/cid10_evit_svs.csv', index=False)

##### Carga de CIDs doenças evitáveis - OCDE

In [ ]:
# # Download de tabela tratada de CIDs evitáveis de acordo com documentos da OCDE
# url = 'https://docs.google.com/spreadsheets/d/1X8AH0zs4PNc_bUpZ3tBvFw2HGpgb9CjShOkwXQ0so_I/edit?usp=sharing'
# # Convert the Google Sheets URL to a CSV export URL
# csv_url = url.replace('/edit?usp=sharing', '/export?format=csv')

# # Read the CSV data into a pandas DataFrame
# ocde_evit = pd.read_csv(csv_url)

# # Now you can work with the DataFrame 'df'
# ocde_evit.head()

,avoid_flag,prevent_flag,treat_flag,Group,Causes of deaths,Rationale for inclusion,Range,cid_pai
0,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,Intestinal diseases,Most of these infections can be prevented thro...,(A00-A09),A0
1,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,"Diphtheria, Tetanus, Poliomyelitis",Most of these infections can be prevented thro...,"(A35, A36, A80)",A35
2,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,"Diphtheria, Tetanus, Poliomyelitis",Most of these infections can be prevented thro...,"(A35, A36, A80)",A36
3,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,"Diphtheria, Tetanus, Poliomyelitis",Most of these infections can be prevented thro...,"(A35, A36, A80)",A80
4,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,Whooping cough,Most of these infections can be prevented thro...,(A37),A37


In [23]:
ocde_evit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   avoid_flag               346 non-null    object
 1   prevent_flag             189 non-null    object
 2   treat_flag               184 non-null    object
 3   Group                    346 non-null    object
 4   Causes of deaths         346 non-null    object
 5   Rationale for inclusion  346 non-null    object
 6   Range                    346 non-null    object
 7   cid_pai                  346 non-null    object
dtypes: object(8)
memory usage: 21.8+ KB


## População Censo

### Carga de População Censo 


Os dados populacionais são os dos Censos de 2010 e 2022.

A fonte de dados para gerar os arquivos ou o endereço do caminho SIDRA, é https://sidra.ibge.gov.br/Tabela/9606.

No caso de municípios, como coletamos variáveis de sexo, raça e faixa etária, o que excedeu os limites do uso do API do SIDRA, então iremos carregar o download feito a posteriori, via arquivo no google drive.

In [ ]:
import gdown
import zipfile
import pandas as pd

# ID of the file on Google Drive
file_id = '1-viTldhMlr7fLakBHF-3psclZjOFV3MN'
# Output file path
output_file = 'files_in_treat/censo_raw.zip'

# Download the file
gdown.download(id=file_id, output=output_file, quiet=False)

# Unzip the file
with zipfile.ZipFile(output_file, 'r') as zip_ref:
    zip_ref.extractall('files_in_treat/unzipped_data')

# Assuming the unzipped file is a CSV, read it into a DataFrame
csv_file_path = 'files_in_treat/unzipped_data/your_csv_file.csv'
df = pd.read_csv(csv_file_path, skiprows=3)

# Save the DataFrame as a Parquet file
df.to_parquet('files_in_treat/data.parquet')


In [71]:
import gdown
# https://drive.google.com/file/d/1-xByhnUTPG25EXlz2Ls2Amnxv5D0id6G/view?usp=drive_link
# ID of the file on Google Drive
file_id = '1-xByhnUTPG25EXlz2Ls2Amnxv5D0id6G'
# Output file path
output_file = 'files_in_treat/censo_2010_2022_raw.csv'

# Download the file
gdown.download(id=file_id, output=output_file, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1-xByhnUTPG25EXlz2Ls2Amnxv5D0id6G
To: /workspaces/heat_br_bloco1/files_in_treat/censo_2010_2022_raw.csv
100%|██████████| 10.5M/10.5M [00:00<00:00, 21.5MB/s]


'files_in_treat/censo_2010_2022_raw.csv'

In [72]:
output_file = 'files_in_treat/censo_2010_2022_raw.csv'
censo_temp = pd.read_csv(output_file, skiprows=3, dtype=str)

tab = censo_temp
print(f'Registros da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 111960
+----+-------+---------+--------+------------------------------------+--------+---------------+---------+----------------+----------------+----------------+----------------+----------------+--------------------+
|    |   Ano | Nível   |   Cód. | Unidade da Federação e Município   | Sexo   | Cor ou raça   |   Total |   75 a 79 anos |   80 a 84 anos |   85 a 89 anos |   90 a 94 anos |   95 a 99 anos | 100 anos ou mais   |
|----+-------+---------+--------+------------------------------------+--------+---------------+---------+----------------+----------------+----------------+----------------+----------------+--------------------|
|  0 |  2010 | UF      |     11 | Rondônia                           | Homens | Branca        |  274670 |           2808 |           1621 |            641 |            224 |             54 | 8                  |
|  1 |  2010 | UF      |     11 | Rondônia                           | Homens | Preta         |   60563 |            672 | 

In [73]:
# Primeira linha de tratamento para salvar arquivo

# Excluir notas e comentários do arquivo
censo_temp01 = censo_temp.dropna(subset=['Total']).reset_index(drop=True)

# Renomear colunas
censo_temp01.rename(columns={'Unidade da Federação': 'unidade_no'}, inplace=True)

# Exibir as primeiras linhas do DataFrame
# censo_temp01.info()

# mostrar dados gerais da tabela após transformações
tab = censo_temp01
print(f'Registros da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 111940
+----+-------+---------+--------+------------------------------------+--------+---------------+---------+----------------+----------------+----------------+----------------+----------------+--------------------+
|    |   Ano | Nível   |   Cód. | Unidade da Federação e Município   | Sexo   | Cor ou raça   |   Total |   75 a 79 anos |   80 a 84 anos |   85 a 89 anos |   90 a 94 anos |   95 a 99 anos | 100 anos ou mais   |
|----+-------+---------+--------+------------------------------------+--------+---------------+---------+----------------+----------------+----------------+----------------+----------------+--------------------|
|  0 |  2010 | UF      |     11 | Rondônia                           | Homens | Branca        |  274670 |           2808 |           1621 |            641 |            224 |             54 | 8                  |
|  1 |  2010 | UF      |     11 | Rondônia                           | Homens | Preta         |   60563 |            672 | 

In [74]:
# Salvar arquivo de população do Censo
censo_temp01.to_parquet('files_in_treat/censo_2010_2022_df.parquet')

### Tratamento de População Censo Municípios

In [4]:
# Carga de arquivo de População do Censo
censo_temp01 = pd.read_parquet('files_in_geral/censo_2010_2022_df.parquet')

In [5]:
# mostrar dados gerais da tabela após transformações
tab = censo_temp01
print(f'Registros da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 111940
+----+-------+---------+--------+------------------------------------+--------+---------------+---------+----------------+----------------+----------------+----------------+----------------+--------------------+
|    |   Ano | Nível   |   Cód. | Unidade da Federação e Município   | Sexo   | Cor ou raça   |   Total |   75 a 79 anos |   80 a 84 anos |   85 a 89 anos |   90 a 94 anos |   95 a 99 anos | 100 anos ou mais   |
|----+-------+---------+--------+------------------------------------+--------+---------------+---------+----------------+----------------+----------------+----------------+----------------+--------------------|
|  0 |  2010 | UF      |     11 | Rondônia                           | Homens | Branca        |  274670 |           2808 |           1621 |            641 |            224 |             54 | 8                  |
|  1 |  2010 | UF      |     11 | Rondônia                           | Homens | Preta         |   60563 |            672 | 

In [6]:
# Identificar a classificação de unidades federativas
print(f'Unidades federativas do arquivo Censo_pop: {censo_temp01.Nível.unique()}')

# Apresentar os anos do Censo
print(f'Anos do arquivo Censo_pop: {censo_temp01.Ano.unique()}')

Unidades federativas do arquivo Censo_pop: ['UF' 'MU']
Anos do arquivo Censo_pop: ['2010' '2022']


In [7]:
# Preparar colunas para conseguir valores da população até 74 anos

# Colunas para transformar em inteiro
cols_age = ['Total','75 a 79 anos', '80 a 84 anos', '85 a 89 anos','90 a 94 anos', '95 a 99 anos', '100 anos ou mais']

# para todas as colunas identificar substituir '-' por 0
censo_temp02 = censo_temp01.copy()
censo_temp02[cols_age] = censo_temp02[cols_age].replace('-',0)

# transformar todas as colunas definidas para formato integer
censo_temp02[cols_age] = censo_temp02[cols_age].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

# mostrar dados gerais da tabela após transformações
tab = censo_temp02
print(f'Registros da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 111940
+----+-------+---------+--------+------------------------------------+--------+---------------+---------+----------------+----------------+----------------+----------------+----------------+--------------------+
|    |   Ano | Nível   |   Cód. | Unidade da Federação e Município   | Sexo   | Cor ou raça   |   Total |   75 a 79 anos |   80 a 84 anos |   85 a 89 anos |   90 a 94 anos |   95 a 99 anos |   100 anos ou mais |
|----+-------+---------+--------+------------------------------------+--------+---------------+---------+----------------+----------------+----------------+----------------+----------------+--------------------|
|  0 |  2010 | UF      |     11 | Rondônia                           | Homens | Branca        |  274670 |           2808 |           1621 |            641 |            224 |             54 |                  8 |
|  1 |  2010 | UF      |     11 | Rondônia                           | Homens | Preta         |   60563 |            672 | 

In [8]:
# Gerar coluna de população total até 74 anos
cols_age_over75 = ['75 a 79 anos', '80 a 84 anos', '85 a 89 anos','90 a 94 anos', '95 a 99 anos', '100 anos ou mais']

censo_temp03 = censo_temp02.copy()

# Subtrair a população de 75 anos ou mais da população total
censo_temp03['pop_under74'] = censo_temp03['Total'] - censo_temp03[cols_age_over75].sum(axis=1)

# Excluir colunar cols_age_over75 e 0 a 4 anos
censo_temp03.drop(columns=cols_age_over75, inplace=True)

# Renomear coluna Total para pop_total
censo_temp03.rename(columns={'Total': 'pop_total', 'Unidade da Federação e Município':'unidade_no','Cód.':'unidade_co' }, inplace=True)

# mostrar dados gerais da tabela após transformações
tab = censo_temp03
print(f'Registros da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 111940
+----+-------+---------+--------------+--------------+--------+---------------+-------------+---------------+
|    |   Ano | Nível   |   unidade_co | unidade_no   | Sexo   | Cor ou raça   |   pop_total |   pop_under74 |
|----+-------+---------+--------------+--------------+--------+---------------+-------------+---------------|
|  0 |  2010 | UF      |           11 | Rondônia     | Homens | Branca        |      274670 |        269314 |
|  1 |  2010 | UF      |           11 | Rondônia     | Homens | Preta         |       60563 |         59252 |
|  2 |  2010 | UF      |           11 | Rondônia     | Homens | Amarela       |       10126 |          9986 |
|  3 |  2010 | UF      |           11 | Rondônia     | Homens | Parda         |      443524 |        437401 |
|  4 |  2010 | UF      |           11 | Rondônia     | Homens | Indígena      |        6143 |          6037 |
+----+-------+---------+--------------+--------------+--------+---------------+------------

In [9]:
# Gerar informação de UF a partir do split de 'Município', com o dividor '('
censo_temp04 = censo_temp03.copy()

censo_temp04['Ano'] = '_' + censo_temp04['Ano']

# censo_pop_temp03['unidade_no'] = censo_pop_temp03['unidade_no'].str.split('(', expand=True)[0] # Permancer o código de UF depois do nome do município

# # renomear coluna unidade_no para uf
# censo_temp04.rename(columns={'unidade_no':'uf','Cód.':'uf_co'}, inplace=True)

# mostrar dados gerais da tabela após transformações
tab = censo_temp04
print(f'Registros da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))


Registros da tabela : 111940
+----+-------+---------+--------------+--------------+--------+---------------+-------------+---------------+
|    | Ano   | Nível   |   unidade_co | unidade_no   | Sexo   | Cor ou raça   |   pop_total |   pop_under74 |
|----+-------+---------+--------------+--------------+--------+---------------+-------------+---------------|
|  0 | _2010 | UF      |           11 | Rondônia     | Homens | Branca        |      274670 |        269314 |
|  1 | _2010 | UF      |           11 | Rondônia     | Homens | Preta         |       60563 |         59252 |
|  2 | _2010 | UF      |           11 | Rondônia     | Homens | Amarela       |       10126 |          9986 |
|  3 | _2010 | UF      |           11 | Rondônia     | Homens | Parda         |      443524 |        437401 |
|  4 | _2010 | UF      |           11 | Rondônia     | Homens | Indígena      |        6143 |          6037 |
+----+-------+---------+--------------+--------------+--------+---------------+------------

In [10]:
# Dividir arquivos em UF e Municípios

# Criar arquivo de UF
censo_uf_temp = censo_temp04[censo_temp04['Nível'] == 'UF'].reset_index(drop=True)

# Criar arquivo de Municípios
censo_mun_temp = censo_temp04[censo_temp04['Nível'] == 'MU'].reset_index(drop=True)

# mostrar dados gerais da tabela após transformações
print(f'Registro da tabela censo_uf_temp: {censo_uf_temp.shape[0]}')
print(f'Registro da tabela censo_mun_temp: {censo_mun_temp.shape[0]}')
# print(tabulate(censo_pop_uf_temp.head(), headers='keys', tablefmt='psql'))

Registro da tabela censo_uf_temp: 540
Registro da tabela censo_mun_temp: 111400


In [11]:
# Preparar para salvar arquivo de UF
censo_uf_temp01 = censo_uf_temp.copy()

# Excluir colunas desnecessárias
censo_uf_temp01.drop(columns=['Nível'], inplace=True)

# Renomear coluna unidade_no para uf_no
censo_uf_temp01.rename(columns={'unidade_no': 'uf_no','unidade_co':'uf_co'}, inplace=True)

censo_uf_temp01['uf_co'] = '_' + censo_uf_temp01['uf_co'].astype(str).str[:2]

# mostrar dados gerais da tabela após transformações
tab = censo_uf_temp01
print(f'Registros da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 540
+----+-------+---------+----------+--------+---------------+-------------+---------------+
|    | Ano   | uf_co   | uf_no    | Sexo   | Cor ou raça   |   pop_total |   pop_under74 |
|----+-------+---------+----------+--------+---------------+-------------+---------------|
|  0 | _2010 | _11     | Rondônia | Homens | Branca        |      274670 |        269314 |
|  1 | _2010 | _11     | Rondônia | Homens | Preta         |       60563 |         59252 |
|  2 | _2010 | _11     | Rondônia | Homens | Amarela       |       10126 |          9986 |
|  3 | _2010 | _11     | Rondônia | Homens | Parda         |      443524 |        437401 |
|  4 | _2010 | _11     | Rondônia | Homens | Indígena      |        6143 |          6037 |
+----+-------+---------+----------+--------+---------------+-------------+---------------+


In [12]:
# Preparar para salvar arquivo de Municipios
censo_mun_temp01 = censo_mun_temp.copy()

# Renomear coluna unidade_no para uf_no
censo_mun_temp01.rename(columns={'unidade_no': 'mun_no','unidade_co': 'mun_co'}, inplace=True)

# Mudar o formato de mun_cod para string e extrair os primeiros 7 dígitos
# censo_mun_temp01['mun_co'] = censo_mun_temp01['mun_co'].astype(str).str[:6]

# Acrescentar um '_' no início do valor da coluna mun_cod
censo_mun_temp01['mun_co'] = '_' + censo_mun_temp01['mun_co'].astype(str).str[:6]

# Criar campo uf com 2 caracteres entre parênteses do campo unidade_co
censo_mun_temp01['uf'] = censo_mun_temp01['mun_no'].str[-3:-1]

# Excluir colunas desnecessárias
censo_mun_temp01.drop(columns=['Nível'], inplace=True)

# Ordenar as colunas
# censo_pop_mun_temp02 = censo_pop_mun_temp01[['Ano','uf_co','UF','mun_cod_sim','mun_no','pop_total','pop_under74']]

# mostrar dados gerais da tabela após transformações
tab = censo_mun_temp01
print(f'Registros da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 111400
+----+-------+----------+----------------------------+--------+---------------+-------------+---------------+------+
|    | Ano   | mun_co   | mun_no                     | Sexo   | Cor ou raça   |   pop_total |   pop_under74 | uf   |
|----+-------+----------+----------------------------+--------+---------------+-------------+---------------+------|
|  0 | _2010 | _110001  | Alta Floresta D'Oeste (RO) | Homens | Branca        |        5311 |          5202 | RO   |
|  1 | _2010 | _110001  | Alta Floresta D'Oeste (RO) | Homens | Preta         |         693 |           673 | RO   |
|  2 | _2010 | _110001  | Alta Floresta D'Oeste (RO) | Homens | Amarela       |          96 |            94 | RO   |
|  3 | _2010 | _110001  | Alta Floresta D'Oeste (RO) | Homens | Parda         |        6323 |          6206 | RO   |
|  4 | _2010 | _110001  | Alta Floresta D'Oeste (RO) | Homens | Indígena      |         233 |           229 | RO   |
+----+-------+----------+----------

In [16]:
# Fazer o merge com nome da UF
censo_mun_temp01['uf_co'] = censo_mun_temp01.mun_co.str[:3]

censo_uf_temp01_ = censo_uf_temp01[['uf_co','uf_no']]

censo_uf_temp02_ = censo_uf_temp01_.drop_duplicates()

print(censo_uf_temp02_.shape)
print(censo_uf_temp02_.head())

(27, 2)
   uf_co     uf_no
0    _11  Rondônia
10   _12      Acre
20   _13  Amazonas
30   _14   Roraima
40   _15      Pará


In [17]:

censo_mun_temp02 = pd.merge(censo_mun_temp01,censo_uf_temp02_,on='uf_co',how='left')

# mostrar dados gerais da tabela após transformações
tab = censo_mun_temp02
print(f'Registros da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 111400
+----+-------+----------+----------------------------+--------+---------------+-------------+---------------+------+---------+----------+
|    | Ano   | mun_co   | mun_no                     | Sexo   | Cor ou raça   |   pop_total |   pop_under74 | uf   | uf_co   | uf_no    |
|----+-------+----------+----------------------------+--------+---------------+-------------+---------------+------+---------+----------|
|  0 | _2010 | _110001  | Alta Floresta D'Oeste (RO) | Homens | Branca        |        5311 |          5202 | RO   | _11     | Rondônia |
|  1 | _2010 | _110001  | Alta Floresta D'Oeste (RO) | Homens | Preta         |         693 |           673 | RO   | _11     | Rondônia |
|  2 | _2010 | _110001  | Alta Floresta D'Oeste (RO) | Homens | Amarela       |          96 |            94 | RO   | _11     | Rondônia |
|  3 | _2010 | _110001  | Alta Floresta D'Oeste (RO) | Homens | Parda         |        6323 |          6206 | RO   | _11     | Rondônia |
|  4 

#### Arquivo pronto - censo por municípios e por UF

In [18]:
# Arquivo de População Censo Tratado (pronto para merge)
censo_uf_temp01.to_parquet('files_in_geral/censo_uf.parquet')

# Salvar arquivo de População do Censo uf
censo_mun_temp02.to_parquet('files_in_geral/censo_mun.parquet')